<a href="https://colab.research.google.com/github/teamgaon/lg_farm/blob/main/220130_sm_plant_doc_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ttach

## 할 일

plant-doc dataset으로 pretrain

대회 데이터로 finetune

b2, b4 비교

In [2]:
!nvidia-smi
# k80 -> T4 -> P100

Sun Jan 30 07:01:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 사용 패키지

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json
import torch
from torch import nn
from torchvision import models
from torch.utils.data import Dataset
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import ttach as tta
import albumentations as A
from albumentations.pytorch import ToTensor
from sklearn.model_selection import KFold

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip /content/drive/MyDrive/LG/plant/train.zip

Archive:  /content/drive/MyDrive/LG/plant/train.zip
   creating: train/
  inflating: train/00009_jpg.rf.b4187ecff7dbde2ac358f15e01adcecb.jpg  
  inflating: train/0000_jpg.rf.16c2413117fefd281d896f29db7789dc.jpg  
  inflating: train/0008003-strawberry-leaf-veiner-large_jpg.rf.d43d5d18783839215677d9155dce471f.jpg  
  inflating: train/000g_jpg.rf.849756b544aa925472b7e9fc46b1bd37.jpg  
  inflating: train/000gb_jpg.rf.0e1c9bf2f8db9b94464f7a1cae4bfcbc.jpg  
  inflating: train/003_JPG_jpg.rf.86fed674bd3d3151c7738a2ae370578e.jpg  
  inflating: train/004-10_jpg.rf.f3aa5b2d87dbcc260ec4895f954ae2c8.jpg  
  inflating: train/0042241_jpg.rf.bcaadbb721fdeb7e52d0638843eeda75.jpg  
  inflating: train/0052_jpg.rf.308bfadd743b49c9ba739e4f83d6c946.jpg  
  inflating: train/0061_jpg.rf.01edf4541c53b5b4015bef2add16f44e.jpg  
  inflating: train/007_JPG_jpg.rf.95eb0dcaf71de6b7d7c6255bf4c6d2ea.jpg  
  inflating: train/009-e1373768789869_jpg.rf.cb71a4a62bbc64926cdc557c5b26a024.jpg  
  inflating: train/00c_jpg.rf

In [6]:
!unzip /content/drive/MyDrive/LG/plant/test.zip

Archive:  /content/drive/MyDrive/LG/plant/test.zip
   creating: test/
  inflating: test/%2320+Bacterial+Spot+and+Speck_jpg.rf.56abfa1b8c07239d7a7b67f36649a105.jpg  
  inflating: test/0000_jpg.rf.7bf8d4c69ad253ee55c87d6e78d1ae28.jpg  
  inflating: test/000_jpg.rf.79b07e9657b862eca6972264b5e2fe0c.jpg  
  inflating: test/00_jpg.rf.7fa2b9652948e8c39a51a68ec5c6b70a.jpg  
  inflating: test/00pe_jpg.rf.42b93f15a0cec3a936d3a236bf71c5a8.jpg  
  inflating: test/01_jpg.rf.9c39f014290d5a331926f19f5c268200.jpg  
  inflating: test/02_-Rust-2017-207u24s_jpg.rf.cb22459400f68cb6d111d18db2f7d834.jpg  
  inflating: test/02c_jpg.rf.8ccc7cfab5597c03b0c2f83f7f0f1df3.jpg  
  inflating: test/039b47d574bc4bb8a14259a1cd96a741_jpg.rf.1f0673225f876af4143d937a5bee973c.jpg  
  inflating: test/03gb_jpg.rf.bd48ab850353cefcfed664c3041abda1.jpg  
  inflating: test/052609%20Hartman%20Crabapple%20scab%20single%20leaf_JPG_jpg.rf.28e5ae89118129c271d86a720d3f5985.jpg  
  inflating: test/0605_Rust-induced_leafspot_jpg.rf.a85

## CustomDataset 제작

In [23]:
train_classes = pd.read_table('/content/drive/MyDrive/LG/plant/train_classes.txt',names=['filename', 'Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf','Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Soybean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot'],sep=',')
test_classes = pd.read_table('/content/drive/MyDrive/LG/plant/test_classes.txt',names=['filename', 'Apple Scab Leaf', 'Apple leaf', 'Apple rust leaf', 'Bell_pepper leaf','Bell_pepper leaf spot', 'Blueberry leaf', 'Cherry leaf', 'Corn Gray leaf spot', 'Corn leaf blight', 'Corn rust leaf', 'Peach leaf', 'Potato leaf', 'Potato leaf early blight', 'Potato leaf late blight', 'Raspberry leaf', 'Soyabean leaf', 'Soybean leaf', 'Squash Powdery mildew leaf', 'Strawberry leaf', 'Tomato Early blight leaf', 'Tomato Septoria leaf spot', 'Tomato leaf', 'Tomato leaf bacterial spot', 'Tomato leaf late blight', 'Tomato leaf mosaic virus', 'Tomato leaf yellow virus', 'Tomato mold leaf', 'Tomato two spotted spider mites leaf', 'grape leaf', 'grape leaf black rot'],sep=',')

display(train_classes)
display(test_classes)

,filename,Apple Scab Leaf,Apple leaf,Apple rust leaf,Bell_pepper leaf,Bell_pepper leaf spot,Blueberry leaf,Cherry leaf,Corn Gray leaf spot,Corn leaf blight,Corn rust leaf,Peach leaf,Potato leaf,Potato leaf early blight,Potato leaf late blight,Raspberry leaf,Soyabean leaf,Soybean leaf,Squash Powdery mildew leaf,Strawberry leaf,Tomato Early blight leaf,Tomato Septoria leaf spot,Tomato leaf,Tomato leaf bacterial spot,Tomato leaf late blight,Tomato leaf mosaic virus,Tomato leaf yellow virus,Tomato mold leaf,Tomato two spotted spider mites leaf,grape leaf,grape leaf black rot
0,2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b9...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,blueberry-leaf-isolated-white-background-blueb...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2323,IMG_8502_JPG_jpg.rf.ff738ad1b405492cbc603a82dd...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2324,role-of-whitefly-in-plant-virus-transmission-b...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2325,grape2b_jpg.rf.ffbc48866d22fdde1f3d6337d73a15d...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2326,tomato-plant-leaves-brown-spots-tomato-leaves-...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


,filename,Apple Scab Leaf,Apple leaf,Apple rust leaf,Bell_pepper leaf,Bell_pepper leaf spot,Blueberry leaf,Cherry leaf,Corn Gray leaf spot,Corn leaf blight,Corn rust leaf,Peach leaf,Potato leaf,Potato leaf early blight,Potato leaf late blight,Raspberry leaf,Soyabean leaf,Soybean leaf,Squash Powdery mildew leaf,Strawberry leaf,Tomato Early blight leaf,Tomato Septoria leaf spot,Tomato leaf,Tomato leaf bacterial spot,Tomato leaf late blight,Tomato leaf mosaic virus,Tomato leaf yellow virus,Tomato mold leaf,Tomato two spotted spider mites leaf,grape leaf,grape leaf black rot
0,glyphosate_jpg.rf.01dea2d24a1e3591855a68e077bb...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,6134794031202304_jpeg_jpg.rf.021531229ad8d669e...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,squashsilver2_JPG_jpg.rf.02bdb1c8a5c8bdf6f689f...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,peach-leaf-16690421_jpg.rf.030bd0c31f731f86e54...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20180511_091133-24l1vhg-e1526047988236_jpg.rf....,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,toma_TMV_jpg.rf.fadfed6938fbeef46c39f78a02da3b...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
235,17fc47_jpg.rf.fe1c3b512f4182d08f00ac2c612ddb1c...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
236,7190f27ead55c0f6e3ff8b982972810a5446a9713a49d_...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
237,636368370164094909-late-blight-15-_jpg.rf.febe...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


In [24]:
def name_to_label(name:str):
  for col in train_classes.columns:
    if train_classes.loc[train_classes['filename'] == name, col].item() == 1:
      return col
  
train_classes['name'] = train_classes['filename'].map(name_to_label)
train_disease = train_classes[['filename','name']]

In [25]:
def name_to_label(name:str):
  for col in test_classes.columns:
    if test_classes.loc[test_classes['filename'] == name, col].item() == 1:
      return col

test_classes['name'] = test_classes['filename'].map(name_to_label)

test_disease = test_classes[['filename','name']]

In [35]:
train_disease

,filename,name
0,2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B...,Tomato leaf late blight
1,Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b9...,Corn leaf blight
2,d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017...,Tomato mold leaf
3,2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2...,Corn leaf blight
4,blueberry-leaf-isolated-white-background-blueb...,Blueberry leaf
...,...,...
2323,IMG_8502_JPG_jpg.rf.ff738ad1b405492cbc603a82dd...,Blueberry leaf
2324,role-of-whitefly-in-plant-virus-transmission-b...,Tomato leaf yellow virus
2325,grape2b_jpg.rf.ffbc48866d22fdde1f3d6337d73a15d...,grape leaf
2326,tomato-plant-leaves-brown-spots-tomato-leaves-...,Tomato leaf bacterial spot


In [34]:
my_dict={}
for i in range(len(train_disease['name'].unique())):
  my_dict[train_disease['name'].unique()[i]] = i

my_dict

{'Apple leaf': 22,
 'Bell_pepper leaf': 10,
 'Bell_pepper leaf spot': 9,
 'Blueberry leaf': 3,
 'Cherry leaf': 24,
 'Corn Gray leaf spot': 21,
 'Corn leaf blight': 1,
 'Corn rust leaf': 5,
 None: 4,
 'Peach leaf': 18,
 'Potato leaf': 27,
 'Potato leaf early blight': 11,
 'Potato leaf late blight': 16,
 'Raspberry leaf': 13,
 'Soyabean leaf': 20,
 'Soybean leaf': 19,
 'Squash Powdery mildew leaf': 15,
 'Strawberry leaf': 23,
 'Tomato Early blight leaf': 17,
 'Tomato Septoria leaf spot': 7,
 'Tomato leaf': 25,
 'Tomato leaf bacterial spot': 12,
 'Tomato leaf late blight': 0,
 'Tomato leaf mosaic virus': 14,
 'Tomato leaf yellow virus': 6,
 'Tomato mold leaf': 2,
 'grape leaf': 26,
 'grape leaf black rot': 8}

In [36]:
def name_to_label(name:str):
  return my_dict[name]

train_disease['name'] = train_disease['name'].map(name_to_label)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [57]:
train_disease['filename'] = train_disease['filename'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [59]:
train_disease

,filename,name
0,2017-08-27%2B-%2BLate%2BBlight%2Bon%2BGypsy%2B...,0
1,Slide1_preview_jpg.rf.00f57cfc188631c10e10e2b9...,1
2,d-to-tylv-fo005-14496A31BE70553130A_jpg.rf.017...,2
3,2-3%20Gray%20leaf%20spot%20BRUCE_jpg.rf.0040c2...,1
4,blueberry-leaf-isolated-white-background-blueb...,3
...,...,...
2323,IMG_8502_JPG_jpg.rf.ff738ad1b405492cbc603a82dd...,3
2324,role-of-whitefly-in-plant-virus-transmission-b...,6
2325,grape2b_jpg.rf.ffbc48866d22fdde1f3d6337d73a15d...,26
2326,tomato-plant-leaves-brown-spots-tomato-leaves-...,12


In [133]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train'):
        self.mode = mode
        self.files = files

    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):

        # image

        image_path = "/content/train/"+train_disease['filename'].iloc[i]
        img = cv2.imread(image_path)
        img = cv2.resize(img, dsize=(416, 416), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32)/255
        img = np.transpose(img, (2,0,1))
        
        if self.mode == 'train':
            # json_path = f'{file}/{file_name}.json'
            # with open(json_path, 'r') as f:
            #     json_file = json.load(f)
            
            # crop = json_file['annotations']['crop']
            # disease = json_file['annotations']['disease']
            # risk = json_file['annotations']['risk']
            
            self.label = train_disease['name'].iloc[i]

            # augmentation = random.randint(0,2)
            # if augmentation==1:
            #     img = img[::-1].copy()
            # elif augmentation==2:
            #     img = img[:,::-1].copy()
            # img = transforms.ToTensor()(img)
            
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                # 'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32),
                'label' : torch.tensor(self.label, dtype=torch.long)
            }
        else:
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                # 'csv_feature' : torch.tensor(csv_feature, dtype=torch.float32)
            }

SyntaxError: ignored

In [125]:
image_path = "/content/train/"+train_disease['filename'].iloc[0]
img = cv2.imread(image_path)
img = cv2.resize(img, dsize=(416, 416), interpolation=cv2.INTER_AREA)
img = img.astype(np.float32)/255
img = np.transpose(img, (2,0,1))

In [126]:
img

array([[[0.4509804 , 0.41960785, 0.3647059 , ..., 0.31764707,
         0.32941177, 0.32156864],
        [0.49803922, 0.4627451 , 0.41568628, ..., 0.2901961 ,
         0.3019608 , 0.30588236],
        [0.5411765 , 0.5176471 , 0.4862745 , ..., 0.2901961 ,
         0.30980393, 0.32156864],
        ...,
        [0.45882353, 0.48235294, 0.49803922, ..., 0.09803922,
         0.09803922, 0.06666667],
        [0.52156866, 0.5372549 , 0.54509807, ..., 0.09803922,
         0.09803922, 0.07058824],
        [0.5803922 , 0.5921569 , 0.5686275 , ..., 0.07450981,
         0.08627451, 0.06666667]],

       [[0.6666667 , 0.63529414, 0.5803922 , ..., 0.6117647 ,
         0.5921569 , 0.5803922 ],
        [0.7058824 , 0.6784314 , 0.6313726 , ..., 0.57254905,
         0.5647059 , 0.5647059 ],
        [0.7372549 , 0.7254902 , 0.69411767, ..., 0.5647059 ,
         0.5686275 , 0.5686275 ],
        ...,
        [0.24705882, 0.27058825, 0.28627452, ..., 0.10196079,
         0.10980392, 0.08235294],
        [0.2

In [127]:
torch.tensor(img, dtype=torch.float32)

tensor([[[0.4510, 0.4196, 0.3647,  ..., 0.3176, 0.3294, 0.3216],
         [0.4980, 0.4627, 0.4157,  ..., 0.2902, 0.3020, 0.3059],
         [0.5412, 0.5176, 0.4863,  ..., 0.2902, 0.3098, 0.3216],
         ...,
         [0.4588, 0.4824, 0.4980,  ..., 0.0980, 0.0980, 0.0667],
         [0.5216, 0.5373, 0.5451,  ..., 0.0980, 0.0980, 0.0706],
         [0.5804, 0.5922, 0.5686,  ..., 0.0745, 0.0863, 0.0667]],

        [[0.6667, 0.6353, 0.5804,  ..., 0.6118, 0.5922, 0.5804],
         [0.7059, 0.6784, 0.6314,  ..., 0.5725, 0.5647, 0.5647],
         [0.7373, 0.7255, 0.6941,  ..., 0.5647, 0.5686, 0.5686],
         ...,
         [0.2471, 0.2706, 0.2863,  ..., 0.1020, 0.1098, 0.0824],
         [0.2902, 0.3098, 0.3176,  ..., 0.0941, 0.0980, 0.0784],
         [0.3412, 0.3529, 0.3373,  ..., 0.0588, 0.0745, 0.0627]],

        [[0.5373, 0.5059, 0.4510,  ..., 0.4000, 0.3922, 0.3804],
         [0.5765, 0.5490, 0.5020,  ..., 0.3647, 0.3647, 0.3647],
         [0.6118, 0.5961, 0.5647,  ..., 0.3569, 0.3686, 0.

# 하이퍼파라미터 및 변수

In [128]:
device = torch.device("cuda:0")
batch_size = 16
class_n = len(my_dict)
learning_rate = 1e-4
embedding_dim = 512
# num_features = len(csv_feature_dict)
max_len = 24*6
dropout_rate = 0.001
epochs = 15
k_folds = 5
vision_pretrain = True
save_path = '/content/drive/MyDrive/LG/plant'

# 데이터셋 구성

In [129]:
train = train_disease['filename']
test = test_disease

labelsss = train_disease['name']
# train, val = train_test_split(train, test_size=0.2, stratify=labelsss)

In [130]:
train_transforms = A.Compose([
A.Normalize(),
A.HorizontalFlip(),
A.VerticalFlip(p=0.5),
ToTensor()
])

In [131]:
train_dataset = CustomDataset(train_disease)
# val_dataset = CustomDataset(val)
test_dataset = CustomDataset(test_disease, mode = 'test')

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
# val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=2, shuffle=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=False)

# 모델

## 이미지 분류 모델 : efficientnet_b7 -> b4 -> b2 -> b4

In [88]:
class CNN_Encoder(nn.Module):
    def __init__(self, class_n, rate=0.1):
        super(CNN_Encoder, self).__init__()
        self.model = models.efficientnet_b2(pretrained=True)
    
    def forward(self, inputs):
        output = self.model(inputs)
        return output

## 시계열 모델 : LSTM

In [ ]:
class RNN_Decoder(nn.Module):
    def __init__(self, max_len, embedding_dim, class_n, rate):
        super(RNN_Decoder, self).__init__()
        self.lstm = nn.LSTM(max_len, embedding_dim)
        self.rnn_fc = nn.Linear(num_features*embedding_dim, 1000)
        self.final_layer = nn.Linear(1000 + 1000, class_n) # resnet out_dim + lstm out_dim
        self.dropout = nn.Dropout(rate)

    def forward(self, enc_out, dec_inp):
        hidden, _ = self.lstm(dec_inp)
        hidden = hidden.view(hidden.size(0), -1)
        hidden = self.rnn_fc(hidden)
        concat = torch.cat([enc_out, hidden], dim=1) # enc_out + hidden 
        fc_input = concat
        output = self.dropout((self.final_layer(fc_input)))
        return output

## 앙상블

In [ ]:
class CNN2RNN(nn.Module):
    def __init__(self, max_len, embedding_dim, class_n, rate):
        super(CNN2RNN, self).__init__()
        self.cnn = CNN_Encoder(embedding_dim, rate)
        self.rnn = RNN_Decoder(max_len, embedding_dim, class_n, rate)
        
    def forward(self, img, seq):
        cnn_output = self.cnn(img)
        output = self.rnn(cnn_output, seq)
        
        return output

In [116]:
model = models.efficientnet_b2(pretrained=True)
model = model.to(device)

# 학습

In [117]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

criterion = nn.CrossEntropyLoss()

In [118]:
def accuracy_function(real, pred):    
    real = real.cpu()
    pred = torch.argmax(pred, dim=1).cpu()
    score = f1_score(real, pred, average='macro')
    return score

def train_step(batch_item, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        score = accuracy_function(label, output)
        return loss, score
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
        score = accuracy_function(label, output)
        return loss, score

In [119]:
import random
import torchvision.transforms as transforms
import gc

In [108]:
gc.collect()

1233

In [132]:
# loss_plot, val_loss_plot = [], []
# metric_plot, val_metric_plot = [], []
      
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)
  
# Start print
print('--------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_dataset)):
  gc.collect()
  loss_plot, val_loss_plot = [], []
  metric_plot, val_metric_plot = [], []
  
  # Print
  print('')
  print(f'FOLD {fold}')
  print('--------------------------------')
  
  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  # Define data loaders for training and testing data in this fold
  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=batch_size, sampler=train_subsampler, num_workers=2)
  val_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=batch_size, sampler=test_subsampler, num_workers=2)
  
  model = models.efficientnet_b2(max_len=max_len, embedding_dim=embedding_dim, class_n=class_n, rate=dropout_rate)
  model = model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                          lr_lambda=lambda epoch: 0.95 ** epoch,
                                          last_epoch=-1,
                                          verbose=False)

  criterion = nn.CrossEntropyLoss()
  
  for epoch in range(epochs):
      total_loss, total_val_loss = 0, 0
      total_acc, total_val_acc = 0, 0
      
      tqdm_dataset = tqdm(enumerate(train_dataloader))
      training = True
      for batch, batch_item in tqdm_dataset:
          batch_loss, batch_acc = train_step(batch_item, training)
          total_loss += batch_loss
          total_acc += batch_acc
          
          tqdm_dataset.set_postfix({
              'Epoch': epoch + 1,
              'Loss': '{:06f}'.format(batch_loss.item()),
              'Mean Loss' : '{:06f}'.format(total_loss/(batch+1)),
              'Mean F-1' : '{:06f}'.format(total_acc/(batch+1))
          })
      loss_plot.append(total_loss/(batch+1))
      metric_plot.append(total_acc/(batch+1))
      
      tqdm_dataset = tqdm(enumerate(val_dataloader))
      training = False
      for batch, batch_item in tqdm_dataset:
          batch_loss, batch_acc = train_step(batch_item, training)
          total_val_loss += batch_loss
          total_val_acc += batch_acc
          
          tqdm_dataset.set_postfix({
              'Epoch': epoch + 1,
              'Val Loss': '{:06f}'.format(batch_loss.item()),
              'Mean Val Loss' : '{:06f}'.format(total_val_loss/(batch+1)),
              'Mean Val F-1' : '{:06f}'.format(total_val_acc/(batch+1))
          })
      val_loss_plot.append(total_val_loss/(batch+1))
      val_metric_plot.append(total_val_acc/(batch+1))
      scheduler.step()

      if np.max(val_metric_plot) == val_metric_plot[-1]:
          torch.save(model.state_dict(), save_path+'best_model'+str(fold)+'.pt')
          print('best')

      

--------------------------------

FOLD 0
--------------------------------


0it [00:00, ?it/s]

520
1232


1it [00:00,  2.01it/s, Epoch=1, Loss=6.987305, Mean Loss=6.987305, Mean F-1=0.000000]

1665


TypeError: ignored

## 적정 에포크 8회인듯?

학습은 한 40분 추론은 한 20분? 걸리는듯

# 학습 결과

In [ ]:
# plt.figure(figsize=(10,7))
# plt.grid()
# plt.plot(loss_plot, label='train_loss')
# plt.plot(val_loss_plot, label='val_loss')
# plt.xlabel('epoch')
# plt.ylabel('loss')
# plt.title("Loss", fontsize=25)
# plt.legend()
# plt.show()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.grid()
# plt.plot(metric_plot, label='train_metric')
# plt.plot(val_metric_plot, label='val_metric')
# plt.xlabel('epoch')
# plt.ylabel('metric')
# plt.title("F-1", fontsize=25)
# plt.legend()
# plt.show()

# 추론

In [ ]:
# transforms = tta.Compose(
#     [
#         tta.HorizontalFlip(),
#         tta.VerticalFlip(),
#         tta.Rotate90(angles=[0, 180]),
#         # tta.Scale(scales=[1, 2, 4]),
#         # tta.Multiply(factors=[0.9, 1, 1.1]),     
#     ]
# )

In [ ]:
def predict(dataset, model0, model1, model2, model3, model4):
    model0.eval()
    model1.eval()
    model2.eval()
    model3.eval()
    model4.eval()
    tqdm_dataset = tqdm(enumerate(dataset))
    results = []
    for batch, batch_item in tqdm_dataset:
        img = batch_item['img'].to(device)
        seq = batch_item['csv_feature'].to(device)
        with torch.no_grad():
            output = 0.2*model0(img, seq) + 0.2*model1(img, seq) + 0.2*model2(img, seq) + 0.2*model3(img, seq) + 0.2*model4(img, seq)
        output = torch.tensor(torch.argmax(output, dim=1), dtype=torch.int32).cpu().numpy()
        results.extend(output)
    return results

model0 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model0.load_state_dict(torch.load(save_path+'best_model'+str(0)+'.pt', map_location=device))
model0.to(device)

model1 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model1.load_state_dict(torch.load(save_path+'best_model'+str(1)+'.pt', map_location=device))
model1.to(device)

model2 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model2.load_state_dict(torch.load(save_path+'best_model'+str(2)+'.pt', map_location=device))
model2.to(device)

model3 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model3.load_state_dict(torch.load(save_path+'best_model'+str(3)+'.pt', map_location=device))
model3.to(device)

model4 = CNN2RNN(max_len=max_len, embedding_dim=embedding_dim, num_features=num_features, class_n=class_n, rate=dropout_rate)
model4.load_state_dict(torch.load(save_path+'best_model'+str(4)+'.pt', map_location=device))
model4.to(device)

preds = predict(test_dataloader, model0, model1, model2, model3, model4)

0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
333it [03:17,  1.74it/s]

In [ ]:
preds = np.array([label_decoder[int(val)] for val in preds])

# 제출 파일 생성

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/LG/sample_submission.csv')
submission['label'] = preds
submission

In [ ]:
submission.to_csv('/content/drive/MyDrive/LG/submission.csv', index=False)

## 할일
파프리카 흰가루병인 식물의 데이터로 초기, 중기, 말기를 학습하는 모델 생성 -> model2

model1이 파프리카 흰가루병이라고 예측하면 해당 index의 testdata를 model2에 투입하고 결과를 반환

3_a9_1